In [2]:

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from time import sleep
from tensorflow.keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [3]:
img_width, img_height = 224, 224  # Default input size for VGG16

img_window = (img_width, img_height, 3)
batch_size =  256

In [4]:
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))  # 3 = number of channels in RGB pictures

In [5]:
import tensorflow as tf
# import keras

print(tf.__version__)
# load our best trained model 
model = tf.keras.models.load_model('model_transferLearningVGG16BS.hdf5')
model.summary()

2.3.0
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 6,423,298
Trainable params: 6,423,298
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Divide a list in to sebsequences of consecutive integers
def subseq_consecutive_nbrs (seq):
    
    current = []
   
    for val in seq:
        
        if current != [] and val != current[-1] + 1:
            yield current
            
            current = []
        current += [val]

    # Test is only necessary because seq might be empty
    if current != []:
        yield current


In [7]:
# Divide the rectangle containing all the persons in small rectangles. such as one resulted rectangle contains only one person
def updated_list_ind(index):

    sub_index = subseq_consecutive_nbrs(index)
    
    list_index = []
    prev = -1000

    petit = False

    for ind in sub_index:
              
        if len(list_index) == 0:
            
            if ind[0]+12 > ind[-1]:
                petit = True
            
            list_index.append((ind[0], ind[-1]))
            prev = ind[-1]
            
        else:
            
            
            if petit :
                list_index[-1] = (list(list_index[-1])[0],ind[-1])            
                prev = ind[-1]

            elif ind[0] <= prev + 30 :
                if prev - list(list_index[-1])[0] <= 30:                  
                    list_index[-1] = (prev,ind[-1])
                else:
                    list_index[-1] = (list(list_index[-1])[0],ind[-1])
                prev = ind[-1]
                
            elif  ind[0]+15 <= ind[-1]:
                list_index.append((ind[0], ind[-1]))
                prev = ind [-1] 
                
            else:
                prev = ind [0] 
            petit = False

    return  list_index, petit 


In [40]:
def predict_faces(img):
    
    img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_NEAREST )
 
    img_tensor = image.img_to_array(img)  # Image data encoded as integers in the 0–255 range
    img_tensor /= 255.  # Normalize to [0,1] for plt.imshow application

    # Extract features
    features = conv_base.predict(img_tensor.reshape(1,img_width, img_height, 3))

    # Make prediction
    try:
        prediction = model.predict(features)
    except:
        prediction = model.predict(features.reshape(1, 7*7*512))
        
    pred = np.argmax(prediction, axis = 1)
    
    if pred == 1: 
        return 1
    
    return 0
    

In [33]:
#Crop the rectangle containing a person in order to get only the coordinates of her face. "Scale" representd the ration face/(face+body)

def crop_rectangle_faces(p0, p01, frame):
    
    for scale in [0.2, 0.3, 0.4, 0.7,1]:
   
        if p0[1] < int(p01[1]*scale): 
        
            # get width almost equal to the lenght 
            lenght  = int(p01[1]*scale) -  p0[1] 
            x = max(int((lenght - (p01[0] - p0[0]))/(-2)) - 10, 0)
            img = frame[  p0[1] : int(p01[1]*scale),   p0[0]+x : max(p01[0]-x,p0[0]+1 )]
            face_label = predict_faces(img)

            if face_label:
                break

    return face_label, (p0[0]+x ,p0[1] ),(max(p01[0]-x,p0[0]+1 ), int(p01[1]*scale)) 


# Background Subtraction Manual to detect motions. 

# In the case of AVDIA Dataset. It's about detecting persons and in particular their faces


In [38]:
def object_moving_detection(video_seq):
    
    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_seq)
    
    # Check if video opened successfully
    if (cap.isOpened()== False):
      print("Error opening video stream or file")
    else:
        _, background = cap.read()
        # Save the first image as background
        background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
        background = cv2.GaussianBlur(background, (7, 7), 0)
        
        plt.imshow(background, cmap='gray')
    
    # Read until video is completed
    while (cap.isOpened()):

        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (7, 7), 0)
           
            diff = cv2.absdiff(gray, background)
          
            
            # Non-Maximum suppression
            lim = 0.50*(np.max(diff)-np.min(diff))
            
            diff[diff<lim]=0
           
            diff = cv2.GaussianBlur(diff, (21, 21), 0)
           
            idxs = np.nonzero(diff)
           
            if len(idxs[0])>1:
                
                x0 = min(idxs[1])
                y0 = min(idxs[0])               
                p0 = tuple([x0, y0])
                
               
                x1 = max(idxs[1])
                y1 = max(idxs[0])              
                p1 = tuple([x1, y1])
                
            
                
                index = np.where(~diff[y0:y1, x0:x1].any(axis=0))[0]
 
                final_list_indexes, small = updated_list_ind(index )

                if len(index)==0 or small :                    
                             
                    face_label, r0, r1 = crop_rectangle_faces(p0, p1, frame)
                    cv2.rectangle(frame, p0, p1, (255, 0, 0), 1)
                    
                    if face_label:
                        cv2.rectangle(frame, r0, r1, (0, 255, 0), 2)
                        
                else :
                    
                    for ind in final_list_indexes :

                        p01 = (ind[0]+x0, y1)                       
                        cv2.rectangle(frame, p0, p01, (0, 0, 255), 1)  
                        
                        face_label, r0, r1 = crop_rectangle_faces(p0, p01, frame)
                      
                        if face_label:
                            cv2.rectangle(frame, r0, r1, (0, 255, 0), 2)
                                                     

                        p0 = (ind[1]+x0, y0)
                        
                    if len( final_list_indexes ) != 0:
                        
                        p11 = (ind[1]+x0, y0)   
                        cv2.rectangle(frame, p11, p1, (0, 0, 255), 1)
                        
                        face_label, r0, r1= crop_rectangle_faces(p11, p1, frame)
                       
                        if face_label:
                            cv2.rectangle(frame, r0, r1, (0, 255, 0), 2)
                            
                            
                        
                          
            # (0, 255, 0) = Green rectangle containing a detected face
            # (0, 0, 255) = Red rectangle referes to the existence of TWO or MORE moving persons in the frame. Each rectangle contains one detected person
            # (255, 0, 0) = Blue rectangle referes to the existence of ONE moving persons in the frame. It contains the detected person
             
            cv2.imshow("Frame", frame)
            

            key = cv2.waitKey(1) & 0xFF

            # if the `q` key is pressed, break from the lop
            if key == ord("q"):
                break
        # Break the loop
        else:
            break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [ ]:
folders = glob.glob(".\AVDIAR_ALL\*")

#select the video 

for folder in folders[4:6]:

    video_seq = glob.glob(folder+"\Video\*1.mp4")  

    object_moving_detection(video_seq[0])